In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset, Subset
import torch.nn.functional as F
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objects as go
import cufflinks as cf
from sklearn.preprocessing import LabelEncoder
cf.go_offline(connected = True)


from mrcnn.visualize import display_instances
from mrcnn import visualize

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import matplotlib.pyplot as plt
from mrcnn.visualize import * 

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import numpy as np
import matplotlib.pyplot as plt

import os
import random
import datetime
import re
import math
import logging
from collections import OrderedDict
import multiprocessing
import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow as tf
import keras
import keras.backend as K
import keras.layers as KL
import keras.engine as KE
import keras.models as KM
import keras.callbacks

from PIL import Image, ImageDraw
from PIL import Image
import skimage 
import pandas as pd
import numpy as np

from mrcnn import utils

tf.compat.v1.disable_eager_execution()

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
class Net(nn.Module):
    def __init__(self,input_dim,output_dim):
        super().__init__()
        
        self.linear_layers = nn.Sequential(nn.Linear(input_dim, 200),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(200,100),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(100,50),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(50,20),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(20,5),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(5, output_dim), 
                                           # 최종 결과는 (20, 3) 이 되므로, dim=-1 로 label 확률값이 들어 있는 마지막 차원을 지정해줘야 함
                                          nn.LogSoftmax(dim = -1)
                                          )
        
    def forward(self,x):
        y = self.linear_layers(x)
        
        return y

In [3]:
class_lst =[
    "short sleeve top",
    "long sleeve top",
    "short sleeve outwear",
    "long sleeve outwear",
    "vest",
    "sling",
    "shorts",
    "trousers",
    "skirt",
    "short sleeve dress",
    "long sleeve dress",
    "vest dress",
    "sling dress",
    'skirt_2'
  ]

In [4]:
model = Net(13, 12)

In [5]:
model.load_state_dict(torch.load('model_1.pt'))

<All keys matched successfully>

In [8]:
items={'short sleeve top': 0, 'long sleeve top': 1, 'short sleeve outwear':2,
       'long sleeve outwear': 3,'vest':4, 'sling':5, 'shorts':6, 'trousers':7,
       'skirt':8, 'short sleeve dress':9, 'long sleeve dress':10,'vest dress':11,'sling dress':12,'skirt_2' : 13}
items

items_2 = np.array(['아메리칸 캐주얼', '캐주얼', '댄디', '포멀', '걸리시', '골프', '홈웨어', '레트로', '로맨틱',
       '스포츠', '스트릿', '시크'])

col = items_2[[8 , 9,  2, 10,  0,  1, 11,  3,  4,  6,  5,  7]]

In [10]:
items

{'short sleeve top': 0,
 'long sleeve top': 1,
 'short sleeve outwear': 2,
 'long sleeve outwear': 3,
 'vest': 4,
 'sling': 5,
 'shorts': 6,
 'trousers': 7,
 'skirt': 8,
 'short sleeve dress': 9,
 'long sleeve dress': 10,
 'vest dress': 11,
 'sling dress': 12,
 'skirt_2': 13}

In [11]:
from PIL import Image
import tqdm

lst_name = []
lst_influ_total = []

directory = 'Total_baseline_data/인플루언서정보_샘플2/'
for files in tqdm.tqdm(os.listdir(directory)):
    
    lst_influ_all = []

    
    for file in os.listdir(directory + files):
        lst_influ = []
        if 'csv' in file:
            df = pd.read_csv(directory + files + '/' + file, index_col=0)
            
            df2 = df.copy()

            df3 = df2[df2['class_name'] != 'Not_Detected'].copy()

            df3['new_class'] = [items[i] if i!= 'skirt_2' else 8 for i in df3.class_name ]

            df3['file_name'] = df3['file_name'] 

            df3['file_name'] = [i.split('_')[0]  for i in df3['file_name'] ]

            df3.index.name = 'index'

            A = []
            for idx,i in enumerate(np.unique(df3.file_name)):
                A.append(df3[df3['file_name'] == i])


            df4 = df3.drop_duplicates('file_name').copy()

            df_zero = pd.DataFrame(np.zeros((df4.shape[0], 13)))
            
            for aa in A:
                data = aa[['score','new_class']].values
            
                for k in data:
                    lst = [0] * 13
                    lst[int(k[1])] = k[0]
                    lst_influ.append(lst)
                    lst_influ_all.append(torch.exp(model(torch.Tensor(lst))).detach().numpy())
                    
                
                    
                    
    lst_influ_total.append(pd.DataFrame(lst_influ_all).sum(axis=0).values)
    lst_name.append(files)


100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [00:02<00:00, 27.35it/s]


In [12]:
df_total = pd.DataFrame(lst_influ_total)
df_total.index = lst_name
df_total.columns = col

In [15]:
df_total.to_excel('style_influencer.xlsx',encoding = 'utf-8-isg')

# BERT 모델 

In [17]:
class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 12)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [22]:
from transformers import BertModel

# 모델 생성
CHECKPOINT_NAME = 'kykim/bert-kor-base'

model_bert = BertModel.from_pretrained(CHECKPOINT_NAME)


Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 12)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [26]:
bert = CustomBertModel(CHECKPOINT_NAME)

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [51]:

A.keys()

dict_keys(['input_ids', 'attention_mask', 'token_type_ids'])

In [69]:
B = bert(A['input_ids'].reshape(1,-1), A['attention_mask'].reshape(1,-1), A['token_type_ids'].reshape(1,-1))

In [70]:
B.max(dim=1)

torch.return_types.max(
values=tensor([1.5197], grad_fn=<MaxBackward0>),
indices=tensor([11]))

In [36]:
CHECKPOINT_NAME = 'kykim/bert-kor-base'
tokenizer_pretrained = CHECKPOINT_NAME

In [46]:
tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)

In [41]:
def preprocessing(sentence,tokenizer):

        # 토큰화 처리
        tokens = tokenizer(
            sentence,                # 1개 문장 
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)           # 2D -> 1D
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask, 
            'token_type_ids': token_type_ids }

In [112]:
bert.load_state_dict(torch.load('bert-kor-base.pth'))

<All keys matched successfully>

In [251]:
sentence = input('원하는 스타일을 입력하시오 : ')

A = preprocessing(sentence= sentence,tokenizer=tokenizer)
B = bert(A['input_ids'].reshape(1,-1), A['attention_mask'].reshape(1,-1), A['token_type_ids'].reshape(1,-1))
C = B.max(dim=1)
style = col[C[1].detach().numpy()][0]
print('해당되는 스타일은 ' + style +' 입니다','\n')

print('#################인플루언서 순위#####################')
lst_img = []
for idx, i in enumerate(df_total[style].sort_values(ascending= False).index[0:10]):
    lst_img.append(i)
    print('\t' +str(idx+1) + '순위 : ' + i)
    
num = np.random.choice(range(len(file_lst)))
file_lst = os.listdir('Total_baseline_data/인플루언서정보_샘플2/' + lst_img[0] + '/images')

num_2 = 0
for idx,i in enumerate(file_lst):
    if ('converted' in i):
        num_2 += 1
        if (num_2 == num):
            img = Image.open('Total_baseline_data/인플루언서정보_샘플2/' + lst_img[0] + '/images/'+i  )
            img.show()
            print(1)


            

원하는 스타일을 입력하시오 : 안녀앟세요
해당되는 스타일은 시크 입니다 

#################인플루언서 순위#####################
	1순위 : @sualboys
	2순위 : @yangkoon__dl
	3순위 : @odor_bubu
	4순위 : @mavlfit
	5순위 : @zxcvr0626
	6순위 : @kj_m.w
	7순위 : @jindalorian
	8순위 : @lil_0uzi_vert
	9순위 : @uuuuk_2_
	10순위 : @so_h_appy
1


In [245]:
img = Image.open('Total_baseline_data/인플루언서정보_샘플2/' + lst_img[0] + '/images/'+i  )
img.show()

In [ ]:
l_baseline_data\인플루언서정보_샘플1\@hotneul\images

In [221]:
num = np.random.choice(range(len(file_lst)))
file_lst = os.listdir('Total_baseline_data/인플루언서정보_샘플2/' + lst_img[-1] + '/images')

num_2 = 0
for idx,i in enumerate(file_lst):
    if ('converted' in i):
        num_2 += 1
        if (num_2 == num):
            print(i)
  

3007463791760602227_0_converted.jpg


In [153]:
for _, i in enumerate(col):
    print(_,i)

0 로맨틱
1 스포츠
2 댄디
3 스트릿
4 아메리칸 캐주얼
5 캐주얼
6 시크
7 포멀
8 걸리시
9 홈웨어
10 골프
11 레트로


In [147]:
C[1]

tensor([4])

In [148]:
for _,i in enumerate(items_2):
    print(_,i)

0 아메리칸 캐주얼
1 캐주얼
2 댄디
3 포멀
4 걸리시
5 골프
6 홈웨어
7 레트로
8 로맨틱
9 스포츠
10 스트릿
11 시크
